<a href="https://colab.research.google.com/github/ysun155/spatial-temporal-data-science-research/blob/main/GSV_image_download.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install google_streetview

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for google-streetview: filename=google_streetview-1.2.9-py3-none-any.whl size=9797 sha256=6914d6e7d94e7350c42acd4bce38bc62a60a346218bca21250020faef8b908fb
  Stored in directory: /root/.cache/pip/wheels/3d/3c/22/66891d135d7de956a903e1eac58515b816e499df0a6f486c28
  Created wheel for kwconfig: filename=kwconfig-1.1.7-py3-none-any.whl size=4991 sha256=18c6d906ccf97acfd457d9997b2bded21c4818405dfe65f4fc07566b3fe96e6c
  Stored in directory: /root/.cache/pip/wheels/75/99/be/6216a0ec525d7bf335628c0ac2463f9a24357c0f07870db419
Successfully built google-streetview kwconfig


In [ ]:
import requests
import json
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import cv2 
import google_streetview.api
import sys
from PIL import Image

In [ ]:
from google.colab import drive 
drive.mount('/content/drive')
%cd /content/drive/My\ Drive/YijiaSun_UCI_Summer_Research

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/YijiaSun_UCI_Summer_Research


In [ ]:
#import google sheets
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

worksheet = gc.open('Cycleway_Metadata').sheet1

# get_all_values gives a list of rows.
rows = worksheet.get_all_values()

# Convert to a DataFrame and render.
import pandas as pd
df = pd.DataFrame.from_records(rows)

In [ ]:
df.columns = df.iloc[0]

df = df.iloc[1:].reset_index(drop=True)

df.head(20)

,,lat,lon,month,panoid,year,num
0,0,33.70716727773588,-117.7653885660456,7.0,XWyA5fFJ4qg7i4q5BKGSOw,2011.0,0
1,1,33.70711142973133,-117.7654166593314,4.0,wzQ5HcOnqiMoVnL2L4U3HQ,2012.0,0
2,2,33.70714406895068,-117.7654189855268,4.0,V2i_zC4V1bTwmozfeTJThg,2015.0,0
3,3,33.70712270762353,-117.7654375950893,11.0,MgAK-XjiYbcHOJmKbDCHXg,2015.0,0
4,4,33.70713336446197,-117.7654021653437,7.0,Xa6YZvYrviiohiMOMg76Vg,2017.0,0
5,5,33.70711953276806,-117.765426053582,12.0,c8lj-qgpn0mnkB0cMkntGQ,2017.0,0
6,6,33.70711285373095,-117.7654052967609,2.0,RbccCCSUmdjiGHHX9lQxkw,2018.0,0
7,7,33.70712201351965,-117.7654488681887,5.0,TJMsBPu6Eep7wNeMekX1MQ,2018.0,0
8,8,33.70711259540437,-117.7654189855268,2.0,xDvDiPbXOqC0PmoR90oFjQ,2021.0,0
9,9,33.70713774332997,-117.7654152278265,6.0,LvSE_czNcW2qvOsfkDg8RA,2022.0,0


In [ ]:
# Import google_streetview for the api module

count = 0
while count < 6477:
  try:
    panoid = df.iloc[count, df.columns.get_loc('panoid')]
    month = int(float(df.iloc[count, df.columns.get_loc('month')]))
    year = int(float(df.iloc[count, df.columns.get_loc('year')]))
    num = df.iloc[count, df.columns.get_loc('num')]
    # Define parameters for street view api
    params = {
    'size': '640x640', # max 640x640 pixels
    'pano': panoid,
    'heading': '0;90;180;270',
    'fov': '90',
    'key': key #here, input your own GSV API key
    }

    # Get a list of all possible queries from multiple parameters
    api_list = google_streetview.helpers.api_list(params)

    # Create a results object for all possible queries
    results = google_streetview.api.results(api_list)

    # Preview results
    #results.preview()

    # Download images to directory 'downloads'
    results.download_links("/content/drive/My Drive/YijiaSun_UCI_Summer_Research/GSV_raw_images")

    #concat images horizontally
    images = [Image.open(x) for x in ['/content/drive/My Drive/YijiaSun_UCI_Summer_Research/GSV_raw_images/gsv_0.jpg', 
                                  '/content/drive/My Drive/YijiaSun_UCI_Summer_Research/GSV_raw_images/gsv_1.jpg', 
                                  '/content/drive/My Drive/YijiaSun_UCI_Summer_Research/GSV_raw_images/gsv_2.jpg', 
                                  '/content/drive/My Drive/YijiaSun_UCI_Summer_Research/GSV_raw_images/gsv_3.jpg']]
    widths, heights = zip(*(i.size for i in images))
    total_width = sum(widths)
    max_height = max(heights)
    new_im = Image.new('RGB', (total_width, max_height))
    x_offset = 0
    for im in images:
      new_im.paste(im, (x_offset,0))
      x_offset += im.size[0]

    new_im.save(f'/content/drive/My Drive/YijiaSun_UCI_Summer_Research/GSV_concated_images/num{num}_y{year}m{month}.jpg')

    #rename all gsv files
    for index in range(4):
      os.rename(f'/content/drive/My Drive/YijiaSun_UCI_Summer_Research/GSV_raw_images/gsv_{index}.jpg', 
              f'/content/drive/My Drive/YijiaSun_UCI_Summer_Research/GSV_raw_images/num{num}_y{year}m{month}_{index}.jpg')
  except FileNotFoundError:
    print('num:', num, 'year:', year, 'month:', month)
    pass
  count += 1